In [182]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Classifiers
from sklearn.svm import SVR
# from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
# from mlxtend.Regressor import StackingCVRegressor #
from mlxtend.classifier import StackingCVClassifier
import xgboost as xgb

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

# Used to ignore warnings generated from StackingCVClassifier
import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

In [183]:
df = pd.read_csv('data.csv')
df.drop([106 , 107], inplace=True)
df = df[df.TYPE!= 'NIL']
X = df.drop('TYPE', axis=1)
y = df[['TYPE']]


In [184]:
X.isna().sum()

IDENTIF      0
RIVER        0
LOCATION     1
ERECTED      0
PURPOSE      0
LENGTH      24
LANES       13
CLEAR-G      2
T-OR-D       3
MATERIAL     0
SPAN        13
REL-L        5
dtype: int64

In [185]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
#list of categorical variables
ordinal_enc_dict = {}

num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include=['object','category']).columns

#This function will encode non-null data and replace it in the original data
for col_name in X:
# Create ordinal encoder for the column
    ordinal_enc_dict[col_name] = OrdinalEncoder()
    # Select the non-null values in the column
    col = X[col_name]
    col_not_null = col[col.notnull()]
    reshaped_vals = col_not_null.values.reshape(-1, 1)
    # Encode the non-null values of the column
    encoded_vals = ordinal_enc_dict[col_name].fit_transform(reshaped_vals)

In [186]:
cat_cols

Index(['IDENTIF', 'RIVER', 'ERECTED', 'PURPOSE', 'LENGTH', 'CLEAR-G', 'T-OR-D',
       'MATERIAL', 'SPAN', 'REL-L'],
      dtype='object')

In [187]:
ordinal_enc_dict = {}

for col_name in X[cat_cols]:
    # Create Ordinal encoder for col
    ordinal_enc_dict[col_name] = OrdinalEncoder()
    col = X[col_name]
    
    # Select non-null values of col
    col_not_null = col[col.notnull()]
    reshaped_vals = col_not_null.values.reshape(-1, 1)
    encoded_vals = ordinal_enc_dict[col_name].fit_transform(reshaped_vals)
    
    # Store the values to non-null values of the column in users
    X.loc[col.notnull(), col_name] = np.squeeze(encoded_vals)

KNN_imputer = KNN()

# Impute and round the users DataFrame
X.iloc[:, :] = np.round(KNN_imputer.fit_transform(X))


X

Imputing row 1/105 with 1 missing, elapsed time: 0.007
Imputing row 101/105 with 0 missing, elapsed time: 0.010


,IDENTIF,RIVER,LOCATION,ERECTED,PURPOSE,LENGTH,LANES,CLEAR-G,T-OR-D,MATERIAL,SPAN,REL-L
0,0.0,1.0,3.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0
1,17.0,0.0,25.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0
2,28.0,0.0,39.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0
3,50.0,0.0,29.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0
4,61.0,1.0,23.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0
5,72.0,0.0,27.0,0.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0
6,83.0,0.0,28.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0
7,94.0,1.0,3.0,0.0,1.0,1.0,2.0,1.0,1.0,0.0,2.0,1.0
8,1.0,0.0,39.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0
9,8.0,0.0,29.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0


In [188]:
# Loop over the column names in users
for col_name in cat_cols:
    print(col_name)
    # Reshape the data
    reshaped = np.round(X[col_name].values).reshape(-1, 1)
    print(f"reshape{reshaped}")
    X[col_name] = ordinal_enc_dict[col_name].inverse_transform(reshaped)
X

IDENTIF
reshape[[  0.]
 [ 17.]
 [ 28.]
 [ 50.]
 [ 61.]
 [ 72.]
 [ 83.]
 [ 94.]
 [  1.]
 [  8.]
 [  9.]
 [ 11.]
 [ 10.]
 [ 12.]
 [ 13.]
 [ 14.]
 [ 15.]
 [ 16.]
 [ 18.]
 [ 19.]
 [ 21.]
 [ 20.]
 [ 22.]
 [ 23.]
 [ 25.]
 [ 24.]
 [ 29.]
 [ 27.]
 [ 26.]
 [ 31.]
 [ 30.]
 [ 33.]
 [ 32.]
 [ 35.]
 [ 34.]
 [ 37.]
 [ 36.]
 [ 38.]
 [ 39.]
 [ 40.]
 [ 41.]
 [ 42.]
 [ 44.]
 [ 43.]
 [ 46.]
 [ 45.]
 [ 47.]
 [ 59.]
 [ 48.]
 [ 99.]
 [ 49.]
 [100.]
 [ 91.]
 [ 52.]
 [ 51.]
 [ 93.]
 [ 54.]
 [ 53.]
 [ 55.]
 [ 57.]
 [ 56.]
 [ 58.]
 [ 60.]
 [ 97.]
 [ 63.]
 [ 62.]
 [ 64.]
 [ 65.]
 [ 67.]
 [ 66.]
 [ 68.]
 [ 73.]
 [ 71.]
 [  2.]
 [ 76.]
 [ 75.]
 [ 69.]
 [ 78.]
 [ 77.]
 [ 74.]
 [ 70.]
 [ 81.]
 [ 80.]
 [ 79.]
 [ 98.]
 [ 82.]
 [  7.]
 [  6.]
 [  5.]
 [  4.]
 [102.]
 [101.]
 [104.]
 [103.]
 [ 85.]
 [ 84.]
 [ 92.]
 [ 86.]
 [  3.]
 [ 87.]
 [ 90.]
 [ 89.]
 [ 88.]
 [ 96.]
 [ 95.]]
RIVER
reshape[[1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]

,IDENTIF,RIVER,LOCATION,ERECTED,PURPOSE,LENGTH,LANES,CLEAR-G,T-OR-D,MATERIAL,SPAN,REL-L
0,E1,M,3.0,CRAFTS,HIGHWAY,MEDIUM,2.0,N,THROUGH,WOOD,SHORT,S
1,E2,A,25.0,CRAFTS,HIGHWAY,MEDIUM,2.0,N,THROUGH,WOOD,SHORT,S
2,E3,A,39.0,CRAFTS,AQUEDUCT,LONG,1.0,N,THROUGH,WOOD,MEDIUM,S
3,E5,A,29.0,CRAFTS,HIGHWAY,MEDIUM,2.0,N,THROUGH,WOOD,SHORT,S
4,E6,M,23.0,CRAFTS,HIGHWAY,MEDIUM,2.0,N,THROUGH,WOOD,MEDIUM,S
5,E7,A,27.0,CRAFTS,HIGHWAY,SHORT,2.0,N,THROUGH,WOOD,MEDIUM,S
6,E8,A,28.0,CRAFTS,AQUEDUCT,MEDIUM,1.0,N,THROUGH,IRON,SHORT,S
7,E9,M,3.0,CRAFTS,HIGHWAY,MEDIUM,2.0,N,THROUGH,IRON,SHORT,S
8,E10,A,39.0,CRAFTS,AQUEDUCT,MEDIUM,1.0,N,DECK,WOOD,MEDIUM,S
9,E11,A,29.0,CRAFTS,HIGHWAY,MEDIUM,2.0,N,THROUGH,WOOD,MEDIUM,S


In [189]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
le = LabelEncoder()

# le = le.inverse_transform
y['TYPE'] = le.fit_transform(y['TYPE'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,stratify=y)
# X_train.head()
# print(le.inverse_transform(y['TYPE']))
# print(y['TYPE'])

In [190]:
num_cols_ind = []
for key in num_features:
    ind = X_train.columns.get_loc(key)
    num_cols_ind.append(ind)
    
cat_cols_ind = []
for key in cat_features:
    ind = X_train.columns.get_loc(key)
    cat_cols_ind.append(ind)

In [191]:
from imblearn.over_sampling import SMOTENC, SVMSMOTE, BorderlineSMOTE, ADASYN
from imblearn.pipeline import make_pipeline as make_pipeline_imb

balancing_smote = SMOTENC(categorical_features=list(cat_cols_ind))
# balancing_smote.fit_resample(X_train, y_train)
from sklearn.preprocessing import LabelEncoder, StandardScaler

smote_nc = SMOTENC(categorical_features=cat_cols_ind, random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train)
X_resampled.head()

,IDENTIF,RIVER,LOCATION,ERECTED,PURPOSE,LENGTH,LANES,CLEAR-G,T-OR-D,MATERIAL,SPAN,REL-L
0,E29,A,26.0,EMERGING,HIGHWAY,MEDIUM,2.0,G,THROUGH,STEEL,MEDIUM,F
1,E18,A,28.0,CRAFTS,RR,MEDIUM,2.0,N,THROUGH,IRON,SHORT,S
2,E94,M,13.0,MATURE,RR,MEDIUM,2.0,G,THROUGH,STEEL,LONG,F
3,E8,A,28.0,CRAFTS,AQUEDUCT,MEDIUM,1.0,N,THROUGH,IRON,SHORT,S
4,E108,A,40.0,MODERN,HIGHWAY,MEDIUM,4.0,G,DECK,STEEL,MEDIUM,S-F


In [192]:
print(X_resampled.shape)
print(X_train.shape)

(174, 12)
(70, 12)


In [193]:
n_estimators = [50,100]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10,15]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [3,5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 3]
# Method of selecting samples for training each tree
bootstrap = [True] 

# {'stackingcvclassifier__randomforestclassifier__bootstrap': True,
#  'stackingcvclassifier__randomforestclassifier__max_depth': 15,
#  'stackingcvclassifier__randomforestclassifier__max_features': 'auto', 
#  'stackingcvclassifier__randomforestclassifier__min_samples_leaf': 1, 
#  'stackingcvclassifier__randomforestclassifier__min_samples_split': 5,
#  'stackingcvclassifier__randomforestclassifier__n_estimators': 100}

In [196]:
pipe_stack.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('standardscaler', StandardScaler(),
                                    Index(['LOCATION', 'LANES'], dtype='object')),
                                   ('onehotencoder',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    Index(['IDENTIF', 'RIVER', 'ERECTED', 'PURPOSE', 'LENGTH', 'CLEAR-G', 'T-OR-D',
          'MATERIAL', 'SPAN', 'REL-L'],
         dtype='object'))])),
  ('stackingcvclassifier',
   StackingCVClassifier(classifiers=(RandomForestClassifier(class_weight='balanced_subsample',
                                                            n_jobs=-1,
                                                            random_state=1000),
                                     KNeighborsClassifier()),
                        meta_classifier=XGBClassifier(base_score=None,
                            

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 

num_features = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include=['object','category']).columns

ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
scaler = StandardScaler()

rf_model = RandomForestClassifier(n_jobs = -1, random_state = 1000, class_weight = "balanced_subsample")
xgb_model = xgb.XGBClassifier()

knn_model = KNeighborsClassifier()

preprocessing = make_column_transformer(
                                        (scaler,num_features)
                                        ,(ohe,cat_features)
                                        ,remainder='passthrough')
# Make pipeline
stacking_model = StackingCVClassifier(classifiers=(rf_model,knn_model), 
                            meta_classifier=xgb_model,
                            random_state=42)


pipe_stack = make_pipeline(preprocessing, stacking_model)

# gbm_param_grid = {'stackingcvclassifier__randomforestclassifier__learning_rate': [0.01,0.1,0.5,0.9],
# 'stackingcvclassifier__randomforestclassifier__n_estimators': [100,200],
# 'stackingcvclassifier__randomforestclassifier__max_depth': [3, 5, 7]}

#      'learning_rate': [0.07, 0.1, 0.3],
# #                 'max_depth': [3, 5, 7],
# #                 'n_estimators': [200, 400, 500]

params = {
               'stackingcvclassifier__randomforestclassifier__n_estimators': n_estimators,  #randomforestregressor__bootstrap
               'stackingcvclassifier__randomforestclassifier__max_features': max_features,
               'stackingcvclassifier__randomforestclassifier__max_depth': max_depth,
               'stackingcvclassifier__randomforestclassifier__min_samples_split': min_samples_split,
               'stackingcvclassifier__randomforestclassifier__min_samples_leaf': min_samples_leaf,
               'stackingcvclassifier__randomforestclassifier__bootstrap': bootstrap,
               'stackingcvclassifier__kneighborsclassifier__n_neighbors':[3,5],
               'stackingcvclassifier__meta_classifier__n_estimators': [10, 50, 100],
               'stackingcvclassifier__meta_classifier__learning_rate': [0.1, 0.5, 1],
               
         }

grid = GridSearchCV(
    verbose=5,#1
    estimator=pipe_stack, 
    param_grid=params, 
    cv=3,
    refit=True
)

 

grid.fit(X_resampled, y_resampled.values.ravel())
# Fit model
# pipe.fit(X_resampled, y_resampled)
# print("Best score : %f" % pipe.score(X_test, y_test.values.ravel()))

Fitting 3 folds for each of 864 candidates, totalling 2592 fits
[CV] stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=10, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifier__min_samples_leaf=1, stackingcvclassifier__randomforestclassifier__min_samples_split=3, stackingcvclassifier__randomforestclassifier__n_estimators=50 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=10, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifier__min_samples_leaf=1, stackingcvclassifier__randomforestclassifier__min_samples_split=3, stackingcvclassifier__randomforestclassifier__n_estimators=50, score=0.707, total=   4.1s
[CV] stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=10, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifier__

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.1s remaining:    0.0s


[CV]  stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=10, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifier__min_samples_leaf=1, stackingcvclassifier__randomforestclassifier__min_samples_split=3, stackingcvclassifier__randomforestclassifier__n_estimators=50, score=0.897, total=   0.8s
[CV] stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=10, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifier__

...

[CV]  stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifier__min_samples_leaf=3, stackingcvclassifier__randomforestclassifier__min_samples_split=3, stackingcvclassifier__randomforestclassifier__n_estimators=50, score=0.690, total=   0.9s
[CV] stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifi

[CV]  stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifier__min_samples_leaf=3, stackingcvclassifier__randomforestclassifier__min_samples_split=5, stackingcvclassifier__randomforestclassifier__n_estimators=50, score=0.948, total=   0.8s
[CV] stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=auto, stackingcvclassifier__randomforestclassifi

[CV]  stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=sqrt, stackingcvclassifier__randomforestclassifier__min_samples_leaf=1, stackingcvclassifier__randomforestclassifier__min_samples_split=3, stackingcvclassifier__randomforestclassifier__n_estimators=100, score=0.914, total=   1.1s
[CV] stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=sqrt, stackingcvclassifier__randomforestclassif

[CV]  stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=sqrt, stackingcvclassifier__randomforestclassifier__min_samples_leaf=3, stackingcvclassifier__randomforestclassifier__min_samples_split=3, stackingcvclassifier__randomforestclassifier__n_estimators=50, score=0.690, total=   0.8s
[CV] stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=sqrt, stackingcvclassifier__randomforestclassifi

[CV]  stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=sqrt, stackingcvclassifier__randomforestclassifier__min_samples_leaf=3, stackingcvclassifier__randomforestclassifier__min_samples_split=5, stackingcvclassifier__randomforestclassifier__n_estimators=50, score=0.948, total=   0.8s
[CV] stackingcvclassifier__kneighborsclassifier__n_neighbors=3, stackingcvclassifier__meta_classifier__learning_rate=0.1, stackingcvclassifier__meta_classifier__n_estimators=10, stackingcvclassifier__randomforestclassifier__bootstrap=True, stackingcvclassifier__randomforestclassifier__max_depth=None, stackingcvclassifier__randomforestclassifier__max_features=sqrt, stackingcvclassifier__randomforestclassifi

In [165]:
print ("score = %3.2f" %(grid.best_score_))
print (grid.best_params_)

score = 0.87
{'stackingcvclassifier__randomforestclassifier__bootstrap': True, 'stackingcvclassifier__randomforestclassifier__max_depth': 15, 'stackingcvclassifier__randomforestclassifier__max_features': 'auto', 'stackingcvclassifier__randomforestclassifier__min_samples_leaf': 1, 'stackingcvclassifier__randomforestclassifier__min_samples_split': 5, 'stackingcvclassifier__randomforestclassifier__n_estimators': 100}


In [180]:
from sklearn.metrics import f1_score

y_pred = pipe.predict(X_test)
f1_score(le.inverse_transform(y_test), y_pred, average='macro')

In [181]:
y_pred

array(['ARCH', 'CANTILEV', 'SIMPLE-T', 'ARCH', 'WOOD', 'SIMPLE-T',
       'SIMPLE-T', 'SIMPLE-T', 'SIMPLE-T', 'SIMPLE-T', 'SIMPLE-T',
       'CANTILEV', 'CONT-T', 'SIMPLE-T', 'SUSPEN', 'SIMPLE-T', 'WOOD',
       'WOOD', 'ARCH', 'CONT-T', 'ARCH', 'WOOD', 'SIMPLE-T', 'SUSPEN',
       'ARCH', 'SIMPLE-T', 'SIMPLE-T', 'SIMPLE-T', 'ARCH', 'SIMPLE-T',
       'WOOD', 'SUSPEN', 'WOOD', 'ARCH', 'CANTILEV'], dtype=object)

In [129]:
pipe.fit(X_train, y_train)
print("Best score : %f" % pipe.score(X_test, y_test))
y_pred = pipe.predict(X_test)
f1_score(y_test, y_pred, average='macro')

C:\Users\deepti\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Best score : 0.685714


0.5011904761904762